# BRONZE LAYER - Ingesting data to Databricks

In [0]:
import requests,json
from pyspark.sql import SparkSession

#defining the Spark Session
spark=SparkSession.builder.appName("JobEL_Bronze").getOrCreate()

#defining the Api key
API_KEY="Shaj1YJ8eNVLWZpkStPamydt"

roles=["Data Engineer","ETL Developer","Spark Engineer","Python Developer","Data Analyst"]
location="India"
all_jobs=[]
for role in roles:
  params = {
  "engine": "google_jobs",
  "q":role,
  "location":location,
  "api_key": API_KEY
  }
  url = "https://www.searchapi.io/api/v1/search"
  response = requests.get(url, params=params)
  jobs=response.json().get("jobs",[])
  for job in jobs:
      job["search_role"]=role
  all_jobs.extend(jobs)
all_jobs

In [0]:
#save json as delta table
df=spark.createDataFrame(all_jobs)
df.write.mode("append").format("delta").saveAsTable("jobs_bronze")
display(df)

# SILVER LAYER - Cleaning and structuring

In [0]:
#cleaning and selecting required columns
df_silver=df.selectExpr(
    "title",
    "description",
    "company_name",
    "location",
    "detected_extensions.posted_at as posted_at",
    "search_role"
).dropna(subset=["title","company_name","location"])
df_silver = df_silver.dropna(subset=["posted_at"])
display(df_silver)

In [0]:
from pyspark.sql.functions import trim, upper, col
df_silver=df_silver.withColumn("company_name",trim(upper(col("company_name"))))
display(df_silver)
df_silver.write.mode("append").format("delta").saveAsTable("jobs_silver")

# GOLD Layer -- Generate KPI tables

In [0]:
from pyspark.sql.functions import col, count

df_gold=df_silver

#KPI 1 - Top Companies
top_companies= df_gold.groupBy("company_name").count().orderBy(col("count").desc())
display(top_companies)

#KPI 2 - Jobs By City
display(df_gold.groupBy("location").agg(count('*').alias("job_count")).orderBy(col("job_count").desc()))

df_gold.write.mode("append").format("delta").saveAsTable("JobAggregator")


In [0]:
#convert dataframe to pandas
import pandas as pdf

pdf= df_gold.orderBy("posted_at",ascending=True).limit(10).toPandas()
display(pdf)

In [0]:
#Creating a plot
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.barh(pdf['company_name'][::-1], pdf['title'][::-1], color='skyblue')
plt.xlabel('Number of jobs')
plt.ylabel('Top Companies Hiring')
plt.grid(True,axis='x',linestyle='--',alpha=0.5)
plt.tight_layout()
plt.show()